## Exploratory Analysis



In [58]:
import os
from glob import glob
import pandas as pd


files = glob("../data/semeval2016/gold/**/*subtask-A*.txt", recursive=True)
files += glob("../data/semeval2016/test/**/*subtask-A*.txt", recursive=True)
files

['../data/semeval2016/gold/dev/100_topics_100_tweets.sentence-three-point.subtask-A.dev.gold.txt',
 '../data/semeval2016/gold/train/100_topics_100_tweets.sentence-three-point.subtask-A.train.gold.txt',
 '../data/semeval2016/gold/devtest/100_topics_100_tweets.sentence-three-point.subtask-A.devtest.gold.txt',
 '../data/semeval2016/test/SemEval2016-task4-test.subtask-A.txt']

In [59]:
from tweepyrate import get_tweets, create_apps

apps = create_apps("../config/gyt-apps.json")

dfs = {
    
}

rets = {
    
}

errors = {
    
}

for file in files:
    print(file)
    dfs[file] = pd.read_table(file, names=["tweet_id", "label"], index_col=0)
    rets[file] = []
    errors[file] = []
    num_success, num_errors = get_tweets(
        apps, list(dfs[file].index),
        tweet_callback= lambda tw_id, response: rets[file].append(response), 
        error_callback = lambda x, y: errors[file].append((x, y))
    )
    print(f"Downloaded tweets {num_success} errors: {num_errors}")

28 apps created
../data/semeval2016/gold/dev/100_topics_100_tweets.sentence-three-point.subtask-A.dev.gold.txt


100%|██████████| 2000/2000 [01:01<00:00, 32.31it/s]


Downloaded tweets 1346 errors: 654
../data/semeval2016/gold/train/100_topics_100_tweets.sentence-three-point.subtask-A.train.gold.txt


100%|██████████| 6000/6000 [03:49<00:00, 26.14it/s]

Downloaded tweets 4056 errors: 1944
../data/semeval2016/gold/devtest/100_topics_100_tweets.sentence-three-point.subtask-A.devtest.gold.txt



100%|██████████| 2000/2000 [01:15<00:00, 26.32it/s]


Downloaded tweets 1397 errors: 603
../data/semeval2016/test/SemEval2016-task4-test.subtask-A.txt


100%|██████████| 31986/31986 [20:04<00:00, 26.56it/s]  


Downloaded tweets 18053 errors: 13933


Let's set the text

In [51]:
for file in files:
    for status in rets[file]:
        dfs[file].loc[status.id, "text"] = status.full_text
    
    print(f"We have {len(rets[file])} non-empty tweets in {file}")
    assert sum(dfs[file]["text"].notna()) == len(rets[file])
    assert sum(dfs[file]["text"].isna()) == len(errors[file])


We have 1346 non-empty tweets in ../data/semeval2016/gold/dev/100_topics_100_tweets.sentence-three-point.subtask-A.dev.gold.txt
We have 4055 non-empty tweets in ../data/semeval2016/gold/train/100_topics_100_tweets.sentence-three-point.subtask-A.train.gold.txt
We have 1397 non-empty tweets in ../data/semeval2016/gold/devtest/100_topics_100_tweets.sentence-three-point.subtask-A.devtest.gold.txt


In [36]:
for file in files:
    print(file)
    print(len(dfs[file]))

caca
2000
caca
2000
caca
2000


In [17]:

def get_lang(file):
    return os.path.splitext(os.path.basename(file))[0]

def load_df(file):
    dialect = get_lang(file)
    
    df = pd.read_table(file, names=["id", "text", "polarity"], index_col=0)
    df["dialect"] = dialect
    
    return df

train_files = glob("../data/tass2020/train/*.tsv")
dev_files = glob("../data/tass2020/dev/*.tsv")
test_files = glob("../data/tass2020/test1.1/*.tsv")

train_dfs = {get_lang(file):load_df(file) for file in train_files}
dev_dfs = {get_lang(file):load_df(file) for file in dev_files}
test_dfs = {get_lang(file):load_df(file) for file in test_files}

train_df = pd.concat(train_dfs.values())
dev_df = pd.concat(dev_dfs.values())
test_df = pd.concat(test_dfs.values())

print(len(train_df), len(dev_df), len(test_df))

train_df.columns, dev_df.columns, test_df.columns

4802 2443 7264


(Index(['text', 'polarity', 'dialect'], dtype='object'),
 Index(['text', 'polarity', 'dialect'], dtype='object'),
 Index(['text', 'polarity', 'dialect'], dtype='object'))

Veamos primero las longitudes (a ver si no hay nada mal cargado)

In [18]:
df = train_df

df["polarity"].value_counts()

N      1885
NEU    1523
P      1394
Name: polarity, dtype: int64

## Ejemplos

In [19]:
pd.options.display.max_colwidth = 200
pd.options.display.max_rows = 100

df[df["polarity"] == "NEU"]

,text,polarity,dialect
id,,,
768529956162924544,@Adriansoler espero y deseo que el interior te cause lo mismo,NEU,es
768557093955698688,"comprendo que te molen mis tattoos, pero no te los hagas 'iguales'",NEU,es
770787189319925760,A mí nunca me podrán hacer una broma porque no cojo llamadas y menos cuando son ocultas,NEU,es
768588061496209408,"@BrujadelTuiters . Estas palabras, del Poema, INSTANTES, son de Nadine Stair. Escritora norteamericana, a la q le gustan los helados.",NEU,es
768506208051494912,@Iluminaughty_ @Spiegel_Spike_ pues me da igual yo escribo lo que pienso ^^ y cuando lo pienso ajajaj si es que lo pienso,NEU,es
...,...,...,...
770138191689093122,Me dijo que no le había gustado no se cuantas cosas pero me estuvo hablando todo el tiempo de eso JAJAJAJJA no entiendo loco,NEU,uy
781329324389597184,Anto te extraño veni a cuidarme,NEU,uy
782750587398000641,Jajajaja mejor que no me la robes Igual vos empezaste!!,NEU,uy


In [20]:
pd.options.display.max_colwidth = 200

df[df["polarity"] == "P"]

,text,polarity,dialect
id,,,
770616744192929792,"Mi última partida jugada, con Sona support. La grandes razones para jugar Sona https://t.co/2rgmMeAFIs",P,es
769959690092642304,Tranquilos que con el.dinero de Camacho seguro mañana empiezan las escabadoras en el Arraijanal,P,es
770590136828559360,"@daniacal aún no, pero si estará jugable en el TGS no creo que tarde demasiado.",P,es
771339035445002240,#feliz septiembre..es bonito retarse..es increible lo mucho que puedes aprender.. medirse con el obstáculo..eres la joyita de la corona,P,es
768577963885522944,Este año el Madrid hará triplete y si lo hace me rapo al cero grabad el mensaje jejej #ChiringuitoPiqué,P,es
...,...,...,...
780770298270474240,@vivianagarcia73 @anitafb Muchas gracias Sandra por tu ayuda y esperamos que los tips sean útiles para Anita. Seguimos en contacto!,P,uy
761932038349946880,@AlmaHistorica doy fe son espectaculares!!,P,uy
795458175348641792,@grisel_ok si hermoso y esta muy agradecido,P,uy


In [29]:
pd.options.display.max_colwidth = 200

df[df["polarity"] == "N"]

,text,polarity
id,,
768512386269638656,@morbosaborealis jajajaja... eso es verdad... aquí no hay uno cuerdo,N
770434744748236800,@ragnomuelle Yo a veces hecho de menos mi pelo largo por eso mismo entre otras cosas,N
768596566378774528,Es conveniente tener un Gobierno democrático que cumpla su función politica. NO lo es tener un Gobierno que representa la #corrupciónPP,N
768071942096166912,"@Jordicanal64 Pues si no fichan ""lo que sea"" además de un ridículo espantoso igual QSF nos deja plantados...esperemos que no",N
772121014436302848,glowen sigue a cuentas inactivas y es en plan oh seguro que esas cuentas te dan mucho más apoyo que a las que le diste unf JAJAJA,N
...,...,...
770952598954373120,@rubenlinoman como todo! Es un jugón! No quiero que lo fichéis. Es el peor día de mi vida,N
770642621966520320,".@DavidCpsberga @goslum Vamos, que usted es el portavoz de 2 millones de personas. Una inmensa... minoría. ¿Le molesta la ""minorización""?",N
771303869896728577,q los GP de La Caixa (y otros) manipulen el Ibex es una bendició gracias a ellos podemos arrañar alg son auténticos PARDILLOS,N
